<img src="https://github.com/nicholasmetherall/digital-earth-pacific-macblue-activities/blob/main/attachments/images/DE_Pacific_banner.JPG?raw=true" width="900"/>

Figure 1.1.a. Jupyter environment + Python notebooks

# Digital Earth Pacific Notebook 1 prepare postcard and load data to csv

The objective of this notebook is to prepare a geomad postcard for your AOI (masking, scaling and loading additional band ratios and spectral indices) and sampling all the datasets into a csv based on your training data geodataframe.

In [1]:
# # This cell is for papermill parameters. DO NOT CHANGE THE VARIABLE NAMES.
# # Default values for manual execution (papermill will override these)
# input_geojson_path = None
# output_csv_path = None

In [2]:
# Parameters
input_geojson_path = "training-data/postcards/suva_east_postcard.geojson"
output_csv_path = "automated_notebook_runs/sampled_data_csvs/suva_east_postcard_sampled_data.csv"


## Step 1.1: Configure the environment

In [3]:
import os
from datetime import datetime
from shapely.geometry import Polygon
from shapely import box
from pyproj import CRS 
import folium
import geopandas as gpd
import numpy as np
import pandas as pd
import rasterio as rio
import xarray as xr
import rioxarray
from ipyleaflet import basemaps
from numpy.lib.stride_tricks import sliding_window_view
import pystac_client
from dask.distributed import Client as DaskClient
from odc.stac import load, configure_s3_access
import planetary_computer
from odc.stac import load
from pystac.client import Client
from skimage.feature import graycomatrix, graycoprops
from utils import load_data, scale, calculate_band_indices, apply_mask, mask_land, mask_deeps, mask_elevation, all_masks, glcm_features, do_prediction

In [4]:
# Reload scripts and imports
%load_ext autoreload
%autoreload 2

In [5]:
# Predefined variable for title and version

# Enter your initials
initials = "nm"

# Enter your site name
site = "yanuca"

# Date
date = datetime.now()

# Make a clean version string
version = f"{initials}-{site}-{date.strftime('%d%m%Y')}"
print(version)

nm-yanuca-27072025


In [6]:
gdfs = []
postcards_path = "training-data/postcards"
file_extension: str = ".geojson"

for filename in os.listdir(postcards_path):
    file_path = os.path.join(postcards_path, filename)
    if os.path.isfile(file_path) and filename.endswith(file_extension):
    # try:
        gdf = gpd.read_file(file_path)
        gdfs.append(gdf)

In [7]:
for filename in os.listdir(postcards_path):
    file_path = os.path.join(postcards_path, filename)
    if os.path.isfile(file_path) and filename.endswith(file_extension):
        print(filename) # This line will print the name of each GeoJSON file
        # The rest of your code to read the file and append to gdfs
        # gdf = gpd.read_file(file_path)
        # gdfs.append(gdf)

print("\nFinished listing GeoJSON files.")

nm-efate-27072025_postcard.geojson
suva_west_postcard.geojson
paunganisu_postcard.geojson
mana-north_postcard.geojson
tulagi_postcard.geojson
serua_2_postcard.geojson
suva_central_postcard.geojson
efate_postcard.geojson
suva_east_postcard.geojson
bootless-bay_postcard.geojson
serua_postcard.geojson
korotogo_postcard.geojson
yanuca_postcard.geojson
mana-south_postcard.geojson

Finished listing GeoJSON files.


## Step 1.2: Configure STAC access and search parameters

In [8]:
catalog = "https://stac.digitalearthpacific.org"
client = Client.open(catalog)

In [9]:
# filename = "nm-efate-27072025_postcard.geojson"

In [10]:
training = gpd.read_file(f"training-data/postcards/{site}_postcard.geojson")
training = training.to_crs("EPSG:4326")
min_lon, min_lat, max_lon, max_lat = training.total_bounds

bbox = [min_lon, min_lat, max_lon, max_lat]

In [11]:
datetime = "2024"

items = client.search(
    collections=["dep_s2_geomad"],
    datetime=datetime,
    bbox=bbox
).item_collection()

print(f"Found {len(items)} items in for {datetime}")

Found 1 items in for 2024


In [12]:
measurements = ["nir", "red", "blue", "green", "emad", "smad", "bcmad", "green", "nir08", "nir09", "swir16", "swir22", "coastal", "rededge1", "rededge2", "rededge3"]
data = load_data(
    items,
    measurements,
    bbox,
)
    
# Now you can use the 'data' variable
print(data)

<xarray.Dataset> Size: 7MB
Dimensions:      (y: 388, x: 505, time: 1)
Coordinates:
  * y            (y) float64 3kB -2.04e+06 -2.04e+06 ... -2.044e+06 -2.044e+06
  * x            (x) float64 4kB 3.049e+06 3.049e+06 ... 3.054e+06 3.054e+06
    spatial_ref  int32 4B 3832
  * time         (time) datetime64[ns] 8B 2024-01-01
Data variables: (12/15)
    nir          (time, y, x) uint16 392kB dask.array<chunksize=(1, 388, 505), meta=np.ndarray>
    red          (time, y, x) uint16 392kB dask.array<chunksize=(1, 388, 505), meta=np.ndarray>
    blue         (time, y, x) uint16 392kB dask.array<chunksize=(1, 388, 505), meta=np.ndarray>
    green        (time, y, x) uint16 392kB dask.array<chunksize=(1, 388, 505), meta=np.ndarray>
    emad         (time, y, x) float32 784kB dask.array<chunksize=(1, 388, 505), meta=np.ndarray>
    smad         (time, y, x) float32 784kB dask.array<chunksize=(1, 388, 505), meta=np.ndarray>
    ...           ...
    swir16       (time, y, x) uint16 392kB dask.array

In [13]:
dask_client = DaskClient(n_workers=1, threads_per_worker=16, memory_limit='16GB')
configure_s3_access(cloud_defaults=True, requester_pays=True)

/srv/conda/envs/notebook/lib/python3.10/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 44053 instead
  warnings.warn(


In [14]:
scaled = scale(data)
scaled = scaled.compute().squeeze()

In [15]:
# Explore the site we are working on
scaled.odc.explore(vmin=0, vmax=0.3, bands=["red", "green", "blue"], crs="EPSG:3832", name=site)

In [16]:
scaled

<xarray.Dataset> Size: 21MB
Dimensions:      (y: 388, x: 505)
Coordinates:
  * y            (y) float64 3kB -2.04e+06 -2.04e+06 ... -2.044e+06 -2.044e+06
  * x            (x) float64 4kB 3.049e+06 3.049e+06 ... 3.054e+06 3.054e+06
    spatial_ref  int32 4B 3832
    time         datetime64[ns] 8B 2024-01-01
Data variables: (12/15)
    nir          (y, x) float64 2MB 0.2194 0.2719 0.3106 ... 0.034 0.0344 0.0311
    red          (y, x) float64 2MB 0.2036 0.2573 0.2764 ... 0.0361 0.0329
    blue         (y, x) float64 2MB 0.1598 0.2052 0.2217 ... 0.0518 0.0486
    green        (y, x) float64 2MB 0.211 0.2669 0.2825 ... 0.0409 0.0413 0.038
    emad         (y, x) float32 784kB 0.2558 0.2923 0.2815 ... 0.09796 0.0906
    smad         (y, x) float32 784kB 2.132e-06 1.584e-06 ... 2.007e-06
    ...           ...
    swir16       (y, x) float64 2MB 0.1002 0.1369 0.1389 ... 0.0334 0.0306
    swir22       (y, x) float64 2MB 0.0625 0.0857 0.0853 ... 0.0311 0.0284
    coastal      (y, x) float64 2MB 0.1227 0.1218 0.1229 ... 0.0543 0.0515
    rededge1     (y, x) float64 2MB 0.2263 0.2882 0.2942 ... 0.0372 0.037 0.034
    rededge2     (y, x) float64 2MB 0.1801 0.2418 0.2412 ... 0.0362 0.0332
    rededge3     (y, x) float64 2MB 0.1911 0.2572 0.259 ... 0.0371 0.037 0.034

In [17]:
scaled = calculate_band_indices(scaled)
Dataset = scaled

### GLCM texture analysis

The objective of this notebook was to train the machine learning model that will allow us to classify an area with land cover classes defined through the training data.

Step 1.2. Input the training data to sample geomad data from the postcard

In [18]:
WINDOW_SIZE = 9
LEVELS = 32

# Input
max = scaled.blue.max().values
min = scaled.blue.min().values
# Scale to 0-LEVELS for GLCM
img = ((scaled.blue - min) / (max - min) * (LEVELS - 1)).clip(0, LEVELS - 1).values.astype(np.uint8)

# Extract overlapping windows
patches = sliding_window_view(img, (WINDOW_SIZE, WINDOW_SIZE))
# Shape: (rows, cols, win_y, win_x)


In [19]:
import numpy as np # Ensure numpy is imported if not already

# Assuming 'patches' is a 4D NumPy array with dimensions (y_coords, x_coords, window_y_size, window_x_size)
# To get the first patch (at y=0, x=0), you would index it like this:
sample_patch_data = patches[0, 0, :, :]

# Verify the shape of the extracted sample patch data
print(f"Shape of sample_patch_data: {sample_patch_data.shape}")

# Call glcm_features directly on this 2D sample data
sample_result = glcm_features(sample_patch_data)

# Print the shape of the result to get the number of features
print(f"Shape of glcm_features output for a single patch: {sample_result.shape}")

Shape of sample_patch_data: (9, 9)
Shape of glcm_features output for a single patch: (7,)


In [20]:
# Use apply_ufunc to vectorize over (row, col) dimensions
result = xr.apply_ufunc(
    glcm_features,
    xr.DataArray(patches, dims=["y", "x", "win_y", "win_x"]),
    input_core_dims=[["win_y", "win_x"]],
    output_core_dims=[["feature"]],
    vectorize=True,
    dask="parallelized",
    output_dtypes=[np.float32]
)

# Add coordinates & names
pad = WINDOW_SIZE - 1
result = result.assign_coords({
    "y": scaled.y[: -pad],
    "x": scaled.x[: -pad],
    "feature": ["contrast", "homogeneity", "energy", "ASM", "correlation", "mean", "entropy"]
})

result_bands = result.to_dataset(dim="feature")

# Combine with original
combined = scaled.copy()
combined = combined.assign(result_bands)

combined

<xarray.Dataset> Size: 45MB
Dimensions:      (y: 388, x: 505)
Coordinates:
  * y            (y) float64 3kB -2.04e+06 -2.04e+06 ... -2.044e+06 -2.044e+06
  * x            (x) float64 4kB 3.049e+06 3.049e+06 ... 3.054e+06 3.054e+06
    spatial_ref  int32 4B 3832
    time         datetime64[ns] 8B 2024-01-01
Data variables: (12/34)
    nir          (y, x) float64 2MB 0.2194 0.2719 0.3106 ... 0.034 0.0344 0.0311
    red          (y, x) float64 2MB 0.2036 0.2573 0.2764 ... 0.0361 0.0329
    blue         (y, x) float64 2MB 0.1598 0.2052 0.2217 ... 0.0518 0.0486
    green        (y, x) float64 2MB 0.211 0.2669 0.2825 ... 0.0409 0.0413 0.038
    emad         (y, x) float32 784kB 0.2558 0.2923 0.2815 ... 0.09796 0.0906
    smad         (y, x) float32 784kB 2.132e-06 1.584e-06 ... 2.007e-06
    ...           ...
    homogeneity  (y, x) float32 784kB 0.6889 0.7028 0.7028 ... nan nan nan
    energy       (y, x) float32 784kB 0.3378 0.3537 0.3548 ... nan nan nan
    ASM          (y, x) float32 784kB 0.1141 0.1251 0.1259 ... nan nan nan
    correlation  (y, x) float32 784kB 0.9229 0.9508 0.9577 ... nan nan nan
    mean         (y, x) float32 784kB 6.625 6.764 6.958 7.306 ... nan nan nan
    entropy      (y, x) float32 784kB 3.729 3.595 3.583 3.9 ... nan nan nan nan

In [21]:
masked_combined, mask = all_masks(combined, return_mask = True)
mask.odc.explore(vmin=0, vmax=0.3, bands=["red", "green", "blue"], crs="EPSG:3832", name=site)

In [22]:
masked_combined.odc.explore(vmin=0, vmax=0.3, bands=["red", "green", "blue"], crs="EPSG:3832", name=site)

/srv/conda/envs/notebook/lib/python3.10/site-packages/odc/geo/_rgba.py:55: RuntimeWarning: invalid value encountered in cast
  return x.astype("uint8")


### Postcard csv

The objective of this notebook was to train the machine learning model that will allow us to classify an area with land cover classes defined through the training data.

Step 1.2. Input the training data to sample geomad data from the postcard

In [23]:
# Reproject training data to the GeoMAD CRS and convert to xarray
training_reprojected = training.to_crs(masked_combined.odc.crs)
training_da = training_reprojected.assign(
    x=training_reprojected.geometry.x, y=training_reprojected.geometry.y
).to_xarray()

# Extract training values from the masked dataset
training_values = (
    masked_combined.sel(training_da[["x", "y"]], method="nearest")
    .squeeze()
    .compute()
    .to_pandas()
)
training_values

,nir,red,blue,green,emad,smad,bcmad,nir08,nir09,swir16,...,homogeneity,energy,ASM,correlation,mean,entropy,y,x,spatial_ref,time
index,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-2039975.0,3053935.0,3832,2024-01-01
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-2041435.0,3051885.0,3832,2024-01-01
2,0.1693,0.1805,0.1583,0.1977,0.202142,1.474273e-06,0.000015,0.2139,0.2122,0.1222,...,0.755556,0.392837,0.154321,0.674530,5.402778,3.549901,-2040975.0,3052335.0,3832,2024-01-01
3,0.2798,0.2439,0.1897,0.2369,0.185207,6.748795e-07,0.000012,0.2140,0.2155,0.1191,...,0.699428,0.312886,0.097897,0.586611,5.423611,4.151052,-2040965.0,3052335.0,3832,2024-01-01
4,0.2939,0.1887,0.1388,0.1767,0.142726,1.400054e-06,0.000011,0.2104,0.2115,0.1174,...,0.642980,0.260023,0.067612,0.575055,5.520833,4.577500,-2040955.0,3052335.0,3832,2024-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-2041645.0,3051145.0,3832,2024-01-01
1004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-2042045.0,3051595.0,3832,2024-01-01
1005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-2042315.0,3052115.0,3832,2024-01-01


In [24]:
# Join the training data with the extracted values and remove unnecessary columns
training_array = pd.concat([training["cc_id"], training_values], axis=1)

# Drop rows where there was no data available
training_array = training_array.dropna()

# Preview our resulting training array
training_array.head()

,cc_id,nir,red,blue,green,emad,smad,bcmad,nir08,nir09,...,homogeneity,energy,ASM,correlation,mean,entropy,y,x,spatial_ref,time
2,2,0.1693,0.1805,0.1583,0.1977,0.202142,1.474273e-06,0.000015,0.2139,0.2122,...,0.755556,0.392837,0.154321,0.674530,5.402778,3.549901,-2040975.0,3052335.0,3832,2024-01-01
3,2,0.2798,0.2439,0.1897,0.2369,0.185207,6.748795e-07,0.000012,0.2140,0.2155,...,0.699428,0.312886,0.097897,0.586611,5.423611,4.151052,-2040965.0,3052335.0,3832,2024-01-01
4,2,0.2939,0.1887,0.1388,0.1767,0.142726,1.400054e-06,0.000011,0.2104,0.2115,...,0.642980,0.260023,0.067612,0.575055,5.520833,4.577500,-2040955.0,3052335.0,3832,2024-01-01
5,2,0.1285,0.0995,0.0891,0.1184,0.183925,1.527652e-06,0.000020,0.1217,0.1301,...,0.643835,0.288006,0.082948,0.585358,5.118055,4.442097,-2040955.0,3052315.0,3832,2024-01-01
6,2,0.1726,0.1808,0.1572,0.1980,0.221409,1.576120e-06,0.000022,0.1209,0.1300,...,0.688317,0.302701,0.091628,0.605075,5.312500,4.215529,-2040965.0,3052325.0,3832,2024-01-01


In [25]:
print(training_array.shape[1], 'total columns')
print('columns included', training_array.columns)

39 total columns
columns included Index(['cc_id', 'nir', 'red', 'blue', 'green', 'emad', 'smad', 'bcmad',
       'nir08', 'nir09', 'swir16', 'swir22', 'coastal', 'rededge1', 'rededge2',
       'rededge3', 'mndwi', 'ndti', 'cai', 'ndvi', 'evi', 'savi', 'ndwi',
       'b_g', 'b_r', 'mci', 'ndci', 'ln_bg', 'contrast', 'homogeneity',
       'energy', 'ASM', 'correlation', 'mean', 'entropy', 'y', 'x',
       'spatial_ref', 'time'],
      dtype='object')


In [26]:
standard_schema = ['cc_id', 'nir', 'red', 'blue', 'green', 'emad', 'smad', 'bcmad',
       'nir08', 'nir09', 'swir16', 'swir22', 'coastal', 'rededge1',
       'rededge2', 'rededge3', 'mndwi', 'ndti', 'cai', 'ndvi', 'evi', 'savi',
       'ndwi', 'b_g', 'b_r', 'mci', 'ndci', 'ln_bg', 'contrast', 'homogeneity',
       'energy', 'ASM', 'correlation', 'mean', 'entropy', 'y', 'x', 'time',
       'spatial_ref']

In [27]:
training_array=training_array[standard_schema]

In [28]:
training_array=training_array.drop(columns=["spatial_ref", "time"])

In [29]:
# Write the training data to a CSV file
training_array.to_csv(f"training-data/csvs/{version}-training.csv", index=False)

In [30]:
training_array["cc_id"].dtype

dtype('int32')